In [1]:
import CS_Indicators as CS
from brix import Indicator, Handler
import PreCompOsmNet
import Simulation
import geopandas as gpd
import pandas as pd

In [2]:
# import sklearn

In [3]:
table_name='epa'

### Load the Saved Data

In [4]:
geogrid=gpd.read_file('tables/{}/geogrid.geojson'.format(table_name))
zones=gpd.read_file('tables/{}/zones.geojson'.format(table_name))
zones['GEOID']=zones['GEOID'].astype(int)
zones=zones.set_index('GEOID')
simpop_df=pd.read_csv('tables/{}/simpop_df.csv'.format(table_name))

### LODES industry is not useful- look up NAICS from wac instead
- TODO: do this in the script which prepares the USA data

In [5]:
import random
naics_cols=[col for col in zones.columns if 'naics' in col]
simpop_df['naics']=0
for ind, row in simpop_df.iterrows():
    if zones.loc[row['work_geoid'], 'emp_total']>0:
        simpop_df.loc[ind, 'naics']=random.choices(naics_cols, weights=zones.loc[row['work_geoid'], naics_cols].values, k=1)[0]
    else:
        simpop_df.loc[ind, 'naics']=random.choices(naics_cols, k=1)[0]

In [6]:
simpop_df=simpop_df.drop('industry', axis=1)

### Create the mobility system

In [7]:
networks, mode_dicts, route_lengths=PreCompOsmNet.create_2_scale_osmnx_network(
    zones.loc[zones['sim_area']], zones.loc[zones['model_area']],
    add_modes=[{'name': 'walk', 'speed': 4800/3600},
               {'name': 'cycle', 'speed': 14000/3600},
               {'name': 'pt', 'speed': 25000/3600}])
modes={mode: Simulation.Mode(mode_dicts[mode]) for mode in mode_dicts}
mob_sys=Simulation.MobilitySystem(modes=modes, networks=networks)

getting internal roads
Buffer : 200
getting external roads
	 Simplified from 1413 to 970 edges and 957 to 607 nodes
	 Pre-computing paths


In [8]:
import numpy as np

class Default_Mode_Choice_model():
    def __init__(self):
        pass
    
    def predict_modes(self, all_trips_df):
        options=['drive', 'cycle', 'walk', 'pt']
        all_trips_df['mode']='drive'
        ind_u1500=all_trips_df['route_distance']<1500
        ind_1500_5k=((all_trips_df['route_distance']>1500)&(all_trips_df['route_distance']<5000))
        ind_5k_10k=((all_trips_df['route_distance']>5000)&(all_trips_df['route_distance']<10000))
        ind_10kplus=all_trips_df['route_distance']>10000

        all_trips_df.loc[ind_u1500==True, 'mode']=np.random.choice(
            options, size=sum(ind_u1500), 
            replace=True, p=[0.1, 0.2, 0.6, 0.1])

        all_trips_df.loc[ind_1500_5k==True, 'mode']=np.random.choice(
            options, size=sum(ind_1500_5k), 
            replace=True, p=[0.25, 0.15, 0.3, 0.3])

        all_trips_df.loc[ind_5k_10k==True, 'mode']=np.random.choice(
            options, size=sum(ind_5k_10k), 
            replace=True, p=[0.6, 0.04, 0.01, 0.35])

        all_trips_df.loc[ind_10kplus==True, 'mode']=np.random.choice(
            options, size=sum(ind_10kplus), 
            replace=True, p=[0.9, 0.01, 0.01, 0.08])
        return all_trips_df

mode_descriptions = [{"name": 'drive',
                    'color': "#e41a1d"},
                     {"name": 'cycle',
                    'color': "#377eb8"},
                     {"name": 'walk',
                    'color': "#4daf4a"},
                     {"name": 'pt',
                    'color': "#ffff33"},
                    ]

### Create and run the Cityscope model

In [9]:
H=Handler(table_name=table_name)
H.reset_geogrid_data()

https://cityio.media.mit.edu/api/tables/list/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/GEOGRID/ Attempt: 0
Proceeding without reviewing GEOGRIDDATA
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
GEOGRIDDATA successfully updated: 9ppgMoPt6nVeJrGBzKefsrhFcyr2UFReaRF7pksagmxQ


In [10]:
m=CS.Mobility_indicator(zones, geogrid, table_name, simpop_df, mob_sys,
                       mode_choice_model=Default_Mode_Choice_model(),
                       mode_descriptions=mode_descriptions,
                       route_lengths=route_lengths)

Init simulation
Getting internal nodes


/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/CS_Indicators.py:619: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids=grid_zones['geometry'].centroid
/opt/anaconda3/envs/cs/lib/python3.9/site-packages/osmnx/distance.py:426: UserWarning: The `get_nearest_nodes` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)


Finding closest nodes to every zone centroid
Computing zone-zone dist mat


In [11]:
d=CS.Density_Indicator(zones=zones)
p=CS.Proximity_Indicator(zones=zones, geogrid=geogrid)


H.add_indicators([
    d, p, 
    m
])

# H.add_indicator(m)

Setting up Proximity Indicator
	 Getting central nodes
	 Finding all reachable geometries from each geometry
	 Calculating baseline scores
https://cityio.media.mit.edu/api/table/epa/GEOGRIDDATA/ Attempt: 0


/opt/anaconda3/envs/cs/lib/python3.9/site-packages/brix/classes.py:981: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  geos = gpd.GeoDataFrame(geos.drop('geometry',1),geometry=geos['geometry'].apply(lambda x: shape(x))) # no crs to avoid warning


Dens Ind: 0:00:00.001319
https://cityio.media.mit.edu/api/table/epa/GEOGRIDDATA/ Attempt: 0
Prox Ind: 0:00:00.014884
Prox HM: 0:00:00.001701
https://cityio.media.mit.edu/api/table/epa/GEOGRIDDATA/ Attempt: 0
Starting MM Update
Schedules and locations
OD
Trip table
Route table
DeckGL


/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/Simulation.py:408: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


Post ABM: <Response [200]>
Post OD: <Response [200]>
Finished MM Update
	 calculation took 0:00:00.143160
	 trips post took 0:00:02.822747


In [12]:
# self=m
# import datetime
# print('Starting MM Update')
# start_calc=datetime.datetime.now()
# new_simpop=self.geogrid_updates(geogrid_data)
# new_simpop_df=pd.DataFrame(new_simpop)
# combined_simpop=self.base_simpop_df.append(new_simpop_df)
# sample_simpop_df=combined_simpop.sample(min(self.N_max, len(combined_simpop)))
# od_output, deckgl_trips, ind=self.simulate(sample_simpop_df)

In [13]:
geogrid_data=H.get_geogrid_data()
print(m.return_indicator(geogrid_data))
print(p.return_indicator(geogrid_data))
print(d.return_indicator(geogrid_data))

https://cityio.media.mit.edu/api/table/epa/GEOGRIDDATA/ Attempt: 0
Starting MM Update
Schedules and locations
OD
Trip table
Route table
DeckGL


/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/Simulation.py:408: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


Post ABM: <Response [200]>
Post OD: <Response [200]>
Finished MM Update
	 calculation took 0:00:00.177965
	 trips post took 0:00:02.347151
{'name': 'Active Mobility', 'raw_value': 0.064, 'value': 0.064, 'ref_value': 0.1, 'viz_type': 'radar'}
Prox Ind: 0:00:00.010384
Prox HM: 0:00:00.001186
{'heatmap': {'type': 'FeatureCollection', 'properties': ['housing', 'employment', 'healthcare', 'hospitality', 'shopping'], 'features': [{'type': 'Feature', 'properties': [0.22448026162111656, 0.5534474780194355, 0.8661016949152542, 0.17541766109785203, 0.028070175438596492], 'geometry': {'type': 'Point', 'coordinates': [-122.13660830090879, 37.478019366165746]}}, {'type': 'Feature', 'properties': [0.22448026162111656, 0.5534474780194355, 0.8661016949152542, 0.17541766109785203, 0.028070175438596492], 'geometry': {'type': 'Point', 'coordinates': [-122.13610583682592, 37.478222536477425]}}, {'type': 'Feature', 'properties': [0.22448026162111656, 0.5534474780194355, 0.8661016949152542, 0.17541766109785

In [ ]:
H.listen()

Table URL: https://cityscope.media.mit.edu/CS_cityscopeJS/?cityscope=epa
Testing indicators
https://cityio.media.mit.edu/api/table/epa/GEOGRIDDATA/ Attempt: 0
Dens Ind: 0:00:00.003377
Prox Ind: 0:00:00.013634
Prox HM: 0:00:00.001432
Starting MM Update
Schedules and locations
OD
Trip table
Route table
DeckGL


/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/Simulation.py:408: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


Post ABM: <Response [200]>
Post OD: <Response [200]>
Finished MM Update
	 calculation took 0:00:00.129374
	 trips post took 0:00:02.811361
Performing initial update
Updating geogrid_data
https://cityio.media.mit.edu/api/table/epa/GEOGRIDDATA/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
Updating table with hash: 9ppgMoPt6nVeJrGBzKefsrhFcyr2UFReaRF7pksagmxQ
https://cityio.media.mit.edu/api/table/epa/GEOGRIDDATA/ Attempt: 0
Dens Ind: 0:00:00.002210
Prox Ind: 0:00:00.011890
Prox HM: 0:00:00.001388
Starting MM Update
Schedules and locations
OD
Trip table
Route table
DeckGL


/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Users/ronandoorley/Documents/Media_Lab/CityScope/CS_Spatial_Modules/Simulation.py:408: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


Post ABM: <Response [200]>
Post OD: <Response [200]>
Finished MM Update
	 calculation took 0:00:00.202527
	 trips post took 0:00:01.922680
Done with update
Local grid hash: 9ppgMoPt6nVeJrGBzKefsrhFcyr2UFReaRF7pksagmxQ
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/me

https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/epa/meta/hashes/ Attemp